<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/RXT3070%E7%9A%84%E9%A0%90%E8%A8%93%E7%B7%B4%E5%92%8C%E5%9C%96%E8%A1%A8%E5%92%8C%E8%B6%85%E5%8F%83%E6%95%B8%E8%AA%BF%E6%95%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""預訓練和PyTorch訓練圖表與超參數調整"""

# ========== 本地環境設置 ==========
# 需提前安裝（在Anaconda Prompt執行）：
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install tensorboard matplotlib

# ========== 導入庫 ==========
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.optim.lr_scheduler import OneCycleLR
from torch.cuda.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import os
import random

# ========== 固定隨機種子 ==========
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # 輸入尺寸固定時可設為True加速

# ========== 超參數配置 ==========
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 256
LR = 0.05
EPOCHS = 150
PATIENCE = 15
MIN_DELTA = 0.001
MODEL_SAVE_PATH = './best_model.pth'  # 本地模型保存路徑
DATA_PATH = './data'                 # 本地數據存儲路徑

# ========== 數據增強配置 ==========
class CIFAR10Enhanced(torchvision.datasets.CIFAR10):
    """增強版數據可視化類別"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def show_augmentation(self, num_samples=4):
        indices = np.random.choice(len(self), num_samples)
        fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))
        for i, idx in enumerate(indices):
            img, label = self[idx]
            img = inv_normalize(img).numpy().transpose((1, 2, 0))
            axes[i].imshow(img)
            axes[i].set_title(f'標籤: {self.classes[label]}')
            axes[i].axis('off')
        plt.show()

# ========== 數據預處理 ==========
CIFAR_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR_STD = (0.2023, 0.1994, 0.2010)

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.15), value='random')
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
])

inv_normalize = transforms.Normalize(
    mean=[-m/s for m, s in zip(CIFAR_MEAN, CIFAR_STD)],
    std=[1/s for s in CIFAR_STD]
)

# ========== 數據加載 ==========
os.makedirs(DATA_PATH, exist_ok=True)

train_dataset = CIFAR10Enhanced(
    root=DATA_PATH, train=True, download=True, transform=train_transform)
test_dataset = CIFAR10Enhanced(
    root=DATA_PATH, train=False, download=True, transform=test_transform)

print("🎨 數據增強可視化：")
train_dataset.show_augmentation()

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4,  # 根據CPU核心數調整
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4,
    pin_memory=True
)

# ========== 修正後的ResNet-18模型 ==========
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block=BasicBlock, num_blocks=[2,2,2,2], num_classes=10):
        super().__init__()
        self.in_planes = 64

        # 修正初始卷積層
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # 權重初始化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # [B,64,16,16]
        x = self.maxpool(x)                  # [B,64,8,8]
        x = self.layer1(x)                   # [B,64,8,8]
        x = self.layer2(x)                   # [B,128,4,4]
        x = self.layer3(x)                   # [B,256,2,2]
        x = self.layer4(x)                   # [B,512,1,1]
        x = self.avgpool(x)                  # [B,512,1,1]
        x = torch.flatten(x, 1)              # [B,512]
        x = self.fc(x)                       # [B,10]
        return x

model = ResNet(num_blocks=[2,2,2,2]).to(DEVICE)
print(f"🔄 模型已加載到 {DEVICE}，參數量：{sum(p.numel() for p in model.parameters()):,}")

# ========== 訓練配置 ==========
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, logits, labels):
        confidence = 1. - self.smoothing
        log_probs = F.log_softmax(logits, dim=-1)
        nll_loss = -log_probs.gather(dim=-1, index=labels.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -log_probs.mean(dim=-1)
        loss = confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
optimizer = SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
scheduler = OneCycleLR(
    optimizer,
    max_lr=0.5,
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.3,
    div_factor=10,
    final_div_factor=100
)
scaler = GradScaler()
writer = SummaryWriter('./logs')  # TensorBoard日誌目錄

# ========== 改進的TTA函數 ==========
def tta_predict(model, images, n_aug=6):
    model.eval()
    with torch.no_grad():
        # 基礎預測
        outputs = model(images)

        # 水平翻轉
        flip_output = model(torch.flip(images, [3]))
        outputs += flip_output

        # 多尺度增強
        for _ in range(n_aug-2):
            aug_images = torch.zeros_like(images)
            for i in range(images.size(0)):
                # 隨機裁剪和縮放
                crop_size = random.randint(24, 32)
                top = random.randint(0, 32 - crop_size)
                left = random.randint(0, 32 - crop_size)
                crop = transforms.functional.resized_crop(
                    images[i], top, left, crop_size, crop_size, 32)
                # 隨機調整亮度
                brightness = random.uniform(0.8, 1.2)
                aug_images[i] = transforms.functional.adjust_brightness(crop, brightness)
            outputs += model(aug_images.to(DEVICE))

    return outputs / n_aug

# ========== 訓練函數 ==========
def train_epoch(epoch):
    model.train()
    total_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE, non_blocking=True)
        labels = labels.to(DEVICE, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_loss += loss.item() * images.size(0)

        # 記錄訓練圖像
        if batch_idx == 0 and epoch % 5 == 0:
            with torch.no_grad():
                images_inv = inv_normalize(images)
                img_grid = torchvision.utils.make_grid(images_inv.cpu())
                writer.add_image(f'訓練圖像/epoch_{epoch}', img_grid, global_step=0)

        # 記錄梯度分布
        if batch_idx % 50 == 0:
            for name, param in model.named_parameters():
                if param.grad is not None:
                    writer.add_histogram(f'梯度/{name}', param.grad, epoch*len(train_loader)+batch_idx)

    return total_loss / len(train_dataset)

# ========== 驗證函數 ==========
def validate(epoch):
    model.eval()
    total_loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = tta_predict(model, images)  # 使用TTA
            loss = criterion(outputs, labels)

            total_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()

    return total_loss / len(test_dataset), correct / len(test_dataset)

# ========== 主訓練循環 ==========
best_acc = 0.0
patience_counter = 0

print("\n🚀 訓練啟動:")
for epoch in range(1, EPOCHS + 1):
    train_loss = train_epoch(epoch)
    val_loss, val_acc = validate(epoch)

    # 記錄指標
    writer.add_scalars('損失曲線', {'訓練損失': train_loss, '驗證損失': val_loss}, epoch)
    writer.add_scalar('準確率/驗證', val_acc, epoch)
    writer.add_scalar('學習率', optimizer.param_groups[0]['lr'], epoch)

    # 打印進度
    print(f"週期 {epoch:03d}/{EPOCHS} | "
          f"訓練損失: {train_loss:.4f} | "
          f"驗證損失: {val_loss:.4f} | "
          f"驗證準確率: {val_acc:.2%} | "
          f"學習率: {optimizer.param_groups[0]['lr']:.2e}")

    # 早停機制
    if val_acc > best_acc + MIN_DELTA:
        best_acc = val_acc
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        patience_counter = 0
        print(f"💾 模型已保存 (準確率 {val_acc:.2%})")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"🛑 連續{PATIENCE}個週期未提升，提前停止訓練")
            break

    # 每週期結束後釋放顯存
    torch.cuda.empty_cache()

writer.close()

# ========== 最終測試 ==========
print("\n🔍 最終測試:")
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
final_loss, final_acc = validate(0)
print(f"🏆 最終測試準確率: {final_acc:.2%}")

# ========== TensorBoard可視化提示 ==========
print("\n🔬 啟動TensorBoard：")
print("在終端執行以下命令：tensorboard --logdir=./logs --port=6006")